In [1]:
cd ..

c:\Users\mwynn\repos\AK-ZtH-building-makemore


In [15]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [3]:
words = open("data/names.txt", "r").read().splitlines()

In [4]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}

In [74]:
# create the dataset
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print("number of examples:", num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

number of examples: 228146


In [77]:
# gradient descent
for k in range(100):

    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float()  # input to the network: one-hot encoding
    logits = xenc @ W  # predict log-counts
    counts = logits.exp()  # counts, equivalent to N
    probs = counts / counts.sum(1, keepdim=True)  # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())

    # backward pass
    W.grad = None  # set to zero the gradient
    loss.backward()

    # update
    W.data += -50 * W.grad

2.4834256172180176
2.4833858013153076
2.4833483695983887
2.4833130836486816
2.4832799434661865
2.4832475185394287
2.4832165241241455
2.483186960220337
2.4831581115722656
2.4831299781799316
2.483103036880493
2.483076333999634
2.48305082321167
2.483025550842285
2.483001232147217
2.4829773902893066
2.482954263687134
2.48293137550354
2.4829089641571045
2.4828872680664062
2.482865571975708
2.4828450679779053
2.4828245639801025
2.482804298400879
2.4827847480773926
2.4827651977539062
2.48274564743042
2.48272705078125
2.482708692550659
2.4826905727386475
2.482672691345215
2.4826550483703613
2.482637405395508
2.4826207160949707
2.4826037883758545
2.4825873374938965
2.4825706481933594
2.4825544357299805
2.482538938522339
2.482523202896118
2.4825077056884766
2.482492446899414
2.4824774265289307
2.4824626445770264
2.482447862625122
2.482433557510376
2.48241925239563
2.482405185699463
2.482390880584717
2.482377052307129
2.48236346244812
2.4823503494262695
2.4823367595672607
2.4823238849639893
2.482

In [ ]:
# sample from the network
for i in range(5):
    
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdim=True)

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

jigua.
sadryrolyiniydavesole.
rish.
be.
ka.
